In [1]:
import modern_robotics as mr
import numpy as np
import numpy.matlib
from calibration import *
numpy.set_printoptions(precision=16)

In [2]:
s = 0.001

w1 = np.array([0,0,1])
p1 = np.array([0,0,0])*s

w2 = np.array([0,-1,0])
p2 = np.array([0,0,0])*s

w3 = np.array([0,-1,0])
p3 = np.array([0,0,-100])*s

w4 = np.array([0,0,-1])
p4 = np.array([-50,250,0])*s

w5 = np.array([0,-1,0])
p5 = np.array([-20,0,-250])*s

w6 = np.array([0,0,-1])
p6 = np.array([-50,250,0])*s



s = 0.001
ax1 = mr.ScrewToAxis(p1,w1,0)
ax2 = mr.ScrewToAxis(p2,w2,0)
ax3 = mr.ScrewToAxis(p3,w3,0)
ax4 = mr.ScrewToAxis(p4,w4,0)
ax5 = mr.ScrewToAxis(p5,w5,0)
ax6 = mr.ScrewToAxis(p6,w6,0)


In [3]:
xi0 = np.array([[0, 0, 0, 0, 0, 0],
    [0, -1, -1, 0, -1, 0],
    [1, 0, 0, -1, 0, -1],

    [0, 0, 0, -0.05, -0.02, -0.05],
    [0, 0, 0, 0.25, 0, 0.25],
    [0, 0, -0.1, 0, -0.25, 0]
])


In [4]:


# xi0 = np.c_[ax1, ax2, ax3, ax4, ax5, ax6]

wist0 = np.array([0,0,0])
pist0 = np.array([250,50,-20])*s

# xist0 = mr.ScrewToAxis(wist0, pist0, 0)

xist0 = np.r_[wist0, pist0]

xi01=np.array([0.0399999800000150,-0.0199999900000075,0.998999500500375,0.0200000000000000,0.0400000000000000,0])
xi02=np.array([0,-1,0,-0.0200000000000000,0,0.0500000000000000])
xi03=np.array([0.178005251232368,-0.984029029284552,-0.00100002950130544,-0.0841845888907446,0.0874136824072620,-100.999920311298])
xi04=np.array([0.0619994730067192,0.0129998895014089,-0.997991517108157,-50.9999969248523,249.000000644789,0.0751505000414993])
xi05=np.array([0.000999959501660306,-0.999999500040373,0,-20.6000000008239,-0.0205991760337826,-249])
xi06=np.array([0.0949994775043106,0.0309998295014066,-0.994994527545148,-51.2730269967030,248.910906980023,2.85959854441601])

xi00=np.c_[xi01,xi02,xi03,xi04,xi05,xi06]
xi00[3:6,:] = xi00[3:6,:]*s

#initial twist
xist00=np.array([0.02, -0.01, 0.01, 249, 51, -20.6])
xist00[3:6] = xist00[3:6]*s

deq1=0
deq2=0.02
deq3=0.002
deq4=0.02

N=10 

P01=np.array([100,0,0])*s
P02=np.array([0,100,0])*s
P03=np.array([0,0,100])*s
PX=np.c_[P01,P02,P03]

P04=np.array([-100,-100,-100])*s

mean_error_before = np.zeros((10,1))
mean_error_traditional = np.zeros((10,1))

max_error_before = np.zeros((10,1))
max_error_traditional = np.zeros((10,1))

In [5]:
gm=np.zeros((4,4,N))
gn=np.zeros((4,4,N))
Pa1=np.zeros((4,N))
Pa2=np.zeros((4,N))
Pa3=np.zeros((4,N))
M_home = np.eye(4)

# cali_joint_angles = np.random.rand(N,6) * 2 * np.pi - np.pi

cali_joint_angles = np.array([[-1.8840,   -0.4296,   -0.5387,    0.8951,    0.6137,    0.0086],
   [ 1.5835,    2.7156,   -2.6493,    0.3922,    0.0345,    1.4384],
   [-1.0787,   -2.4707,    1.4289,    2.5833,   -2.0796,   -0.1758],
   [-0.2531,   -3.0061,    2.2252,    2.0527,    1.9451,    2.6913],
   [-0.2537,    1.1411,   -0.9818,   -0.8954,    0.9576,    2.8640],
   [-1.0407,    1.4736,    2.3483,    1.6613,    0.2297,    1.0713],
   [ 1.2141,   -1.6173,    2.0243,    1.4716,    2.4530,    0.0806],
   [ 0.7989,    2.3981,   -1.9352,   -2.6370,   -2.7984,    1.8965],
   [-2.4100,    0.5888,    1.0122,    0.0538,   -1.3132,   -1.7416],
   [-2.7834,    0.5558,   -2.6076,   -1.2264,    0.4552,    2.4169]])


for i in range(0,N):
    
   Pa1[:,i] = np.dot(mr.FKinSpace(se3Exp(xist00),xi00,cali_joint_angles[i,:]), np.r_[P01,1].reshape((4,1)) ).reshape((4,)) # + ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
   Pa2[:,i] = np.dot(mr.FKinSpace(se3Exp(xist00),xi00,cali_joint_angles[i,:]), np.r_[P02,1].reshape((4,1)) ).reshape((4,)) # + ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
   Pa3[:,i] = np.dot(mr.FKinSpace(se3Exp(xist00),xi00,cali_joint_angles[i,:]), np.r_[P03,1].reshape((4,1)) ).reshape((4,)) # + ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
   PY = np.c_[Pa1[0:3,i],Pa2[0:3,i],Pa3[0:3,i]]

   R,t = Registration(PX, PY)

   gm[0:3,0:3,i] = R
   gm[0:3,3,i] = t
   gn[:,:,i] = mr.FKinSpace(se3Exp(xist00),xi00,cali_joint_angles[i,:])


In [7]:
xiTFP, xistTFP = traditionalCalibrationPuma(xi0, xist00, cali_joint_angles, gm, 10)

In [8]:
Nt=40
error_before=np.zeros((Nt,1))
error_afterMinimal=np.zeros((Nt,1))
error_afterTraditional=np.zeros((Nt,1))
# test_joint_config = np.c_[np.random.rand(Nt,1) * 2 * np.pi, np.random.rand(Nt,1) * 2 * np.pi, np.random.rand(Nt,1) * 2 * np.pi, np.random.rand(Nt,1) * 2 * np.pi, np.random.rand(Nt,1) * 2 * np.pi, np.random.rand(Nt,1) * 2 * np.pi]
test_joint_config = np.array([[0.9638,    4.7010,    0.7958,    0.7878,    5.4706,    3.6449],
    [4.1730,    0.2150,    4.2986,    1.5148,    2.6468,    3.2169],
    [1.1238,    3.1591,    4.4531,    3.0308,    0.2344,    1.9784],
    [5.5455,    2.3617,    4.5584,    5.7675,    4.8890,    2.0374],
    [0.0764,    5.6123,    5.2309,    1.7443,    2.0193,    0.4533],
    [0.3618,    1.1489,    3.7722,    3.9826,    3.9629,    2.7299],
    [4.8365,    0.7277,    6.1300,    3.7114,    4.4512,    3.7307],
    [0.2908,    2.5281,    5.6895,    2.7331,    5.5883,    2.1709],
    [5.5061,    4.2804,    0.3817,    5.2274,    2.7168,    3.1228],
    [4.8305,    4.7148,    4.7300,    0.7391,    6.0528,    2.5789],
    [0.6135,    4.5095,    3.1722,    2.9321,    3.4346,    1.7474],
    [2.7995,    4.6050,    1.1352,    2.0566,    4.3107,    5.3000],
    [3.6483,    1.9823,    0.1329,    1.2445,    3.2064,    0.3195],
    [3.8132,    4.2129,    6.1100,    5.0814,    0.3811,    4.6040],
    [1.2981,    0.7167,    0.4081,    5.2829,    2.0184,    5.9719],
    [0.9647,    5.9641,    4.3064,    4.6935,    2.1360,    2.0369],
    [3.1787,    3.0045,    5.7506,    4.9671,    3.5788,    5.4392],
    [3.0397,    3.7408,    4.6048,    5.8513,    1.3934,    1.9979],
    [0.7725,    3.7729,    5.2734,    5.0273,    3.2415,    1.5234],
    [1.8844,    2.4898,    5.0324,    4.5627,    5.6261,    1.4856],
    [0.8552,    0.5339,    0.6666,    3.9477,    1.3836,    2.2347],
    [0.3298,    1.6431,    4.4923,    0.1758,    2.8066,    1.1581],
    [2.4148,    1.6654,    6.1375,    2.4451,    3.0595,    3.5456],
    [4.6470,    3.1660,    0.5725,    1.0201,    5.0876,    5.2989],
    [4.1210,    1.3379,    5.2131,    1.8522,    6.2609,    2.8180],
    [6.0205,    2.2244,    1.7581,    4.2075,    2.6181,    3.2411],
    [3.1747,    1.2239,    0.3086,    5.5310,    0.3393,    4.5276],
    [3.9955,    3.2007,    3.4194,    0.4378,    5.6101,    4.5167],
    [2.1741,    4.3278,    4.9945,    0.8397,    5.6909,    0.3497],
    [1.5464,    2.8441,    3.3988,    3.4867,    0.7821,    2.5216],
    [5.9767,    6.0693,    1.3174,    5.6443,    1.3256,    5.5995],
    [1.0392,    4.7868,    1.8845,    0.0695,    3.3875,    3.4622],
    [3.3143,    0.0839,    3.7409,    3.1029,    3.1916,    2.5105],
    [6.2641,    5.6654,    4.2558,    2.8627,    3.4595,    6.1946],
    [5.7134,    3.2292,    3.7496,    5.3616,    0.1179,    4.8911],
    [4.0955,    3.9403,    1.9797,    4.0581,    4.7512,    5.9763],
    [0.8259,    4.5248,    5.5307,    1.9324,    2.7693,    3.8078],
    [4.5514,    0.1548,    6.0833,    0.3625,    2.1999,    1.5220],
    [3.9793,    4.2085,    5.7837,    0.7791,    5.5535,    3.5919],
    [5.0110,    4.7847,    2.2131,    5.9161,    6.0310,    4.4978]])

for i in range(0,Nt):
    
    error_before[i] = np.linalg.norm( np.dot( mr.FKinSpace(se3Exp(xist0),xi0, test_joint_config[i,:]) - mr.FKinSpace(se3Exp(xist00),xi00,test_joint_config[i,:]), np.r_[P04,1])) / s
    
    error_afterTraditional[i] = np.linalg.norm( np.dot( mr.FKinSpace(se3Exp(xistTFP), xiTFP, test_joint_config[i,:]) - mr.FKinSpace(se3Exp(xist00),xi00,test_joint_config[i,:]), np.r_[P04,1]) ) / s


In [9]:
mean_error_before[0,0] = np.mean(error_before)
mean_error_traditional[0,0] = np.mean(error_afterTraditional)


In [10]:
mean_error_before

array([[47.31496334638915],
       [ 0.              ],
       [ 0.              ],
       [ 0.              ],
       [ 0.              ],
       [ 0.              ],
       [ 0.              ],
       [ 0.              ],
       [ 0.              ],
       [ 0.              ]])

In [11]:
mean_error_traditional

array([[0.1197706595050253],
       [0.                ],
       [0.                ],
       [0.                ],
       [0.                ],
       [0.                ],
       [0.                ],
       [0.                ],
       [0.                ],
       [0.                ]])

In [23]:
np.round(mr.FKinSpace(se3Exp(xistTFP), np.round(xiTFP,4), test_joint_config[i,:]),4)

array([[ 0.815 , -0.55  , -0.1824,  0.0478],
       [ 0.5501,  0.6354,  0.542 , -0.1301],
       [-0.1822, -0.542 ,  0.8204, -0.0103],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

In [24]:
np.round(mr.FKinSpace(se3Exp(xist00),np.round(xi00,4),test_joint_config[i,:]),4)

array([[ 0.815 , -0.55  , -0.1824,  0.0479],
       [ 0.5501,  0.6354,  0.542 , -0.1301],
       [-0.1822, -0.542 ,  0.8204, -0.0102],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

In [25]:
se3Exp(xistTFP)

array([[ 0.9999000050128466, -0.0100989937984625, -0.009899004978681 ,
         0.2488373212697363],
       [ 0.0098990037953073,  0.9997500125037125, -0.0200479979720457,
         0.0524387209328431],
       [ 0.0100989949584019,  0.0199480029848923,  0.9997500124919955,
        -0.0188359212872217],
       [ 0.                ,  0.                ,  0.                ,
         1.                ]])

In [6]:
def traditionalCalibrationPuma(xi0, xist0, vtheta, gm, M):
   
    xi = np.copy(xi0)

    xist = np.copy(xist0)

    M_home = np.eye(4)

    N = np.size(vtheta,0)

    gn = np.zeros((4,4,N))
    dg = np.zeros((4,4,N))
    vLog = np.zeros((6,N))

    for m in range(0,M):

        for i in range(0,N):


            gn[:,:,i]= mr.FKinSpace(se3Exp(xist),xi,vtheta[i,:])
            dg[:,:,i] = np.linalg.solve(gn[:,:,i].T, gm[:,:,i].T).T
            vLog[:,i]= vlog(dg[:,:,i])

        simY = np.zeros((600, 1))

        for i in range(1, N+1):

            simY[6*i - 6: 6*i, 0] = vLog[:,i-1]


        simJ = np.zeros((600,42))



        for k in range(0,N):

            mat1 = se3Exp( np.dot(vtheta[k,0], xi[:,0]))
            a_mat1 = aMatrix(xi[:,0], vtheta[k,0])

            mat2 = se3Exp( np.dot(vtheta[k,1], xi[:,1]))
            a_mat2 = aMatrix(xi[:,1], vtheta[k,1])

            mat3 = se3Exp( np.dot(vtheta[k,2], xi[:,2]))
            a_mat3 = aMatrix(xi[:,2], vtheta[k,2])

            mat4 = se3Exp( np.dot(vtheta[k,3], xi[:,3]))
            a_mat4 = aMatrix(xi[:,3], vtheta[k,3])

            mat5 = se3Exp( np.dot(vtheta[k,4], xi[:,4]))
            a_mat5 = aMatrix(xi[:,4], vtheta[k,4])

            mat6 = se3Exp( np.dot(vtheta[k,5], xi[:,5]))
            a_mat6 = aMatrix(xi[:,5], vtheta[k,5])

            a_mat_st = aMatrixST(xist)

            simJ[6*k:6+6*k,0:6  ] = a_mat1
            simJ[6*k:6+6*k,6:12 ] = np.dot(mr.Adjoint(mat1), a_mat2)
            simJ[6*k:6+6*k,12:18] = np.dot(mr.Adjoint(np.linalg.multi_dot([mat1, mat2])), a_mat3)
            simJ[6*k:6+6*k,18:24] = np.dot(mr.Adjoint(np.linalg.multi_dot([mat1, mat2, mat3])), a_mat4)
            simJ[6*k:6+6*k,24:30] = np.dot(mr.Adjoint(np.linalg.multi_dot([mat1, mat2, mat3, mat4])), a_mat5)
            simJ[6*k:6+6*k,30:36] = np.dot(mr.Adjoint(np.linalg.multi_dot([mat1, mat2, mat3, mat4, mat5])), a_mat6)
            simJ[6*k:6+6*k,36:42] = np.dot(mr.Adjoint(np.linalg.multi_dot([mat1, mat2, mat3, mat4, mat5, mat6])), a_mat_st)



        dp = np.dot(np.linalg.pinv(simJ),simY)


        dp = dp.reshape((len(dp),))


        xi[:,0] = (xi[:,0].reshape((6,1))+dp[0:6].reshape((6,1))).reshape((6,))
        xi[0:3,0] = xi[0:3,0] / np.linalg.norm(xi[0:3,0])
        xi[3:6,0] = xi[3:6,0] + np.dot( np.dot(xi[0:3,1].T, xi[3:6,0]) /  np.dot(xi[0:3,0].T, xi[0:3,0] ), xi[0:3,0])

        xi[:,1] = (xi[:,1].reshape((6,1)) + dp[6:12].reshape((6,1))).reshape((6,))
        xi[0:3,1] = xi[0:3,1]/np.linalg.norm(xi[0:3,1])
        xi[3:6,1] = xi[3:6,1] - np.dot( np.dot(xi[0:3,1].T, xi[3:6,1]) / np.dot(xi[0:3,1].T,xi[0:3,1]), xi[0:3,1])

        xi[:,2] = (xi[:,2].reshape((6,1)) + dp[12:18].reshape((6,1))).reshape((6,))
        xi[0:3,2] = xi[0:3,2]/np.linalg.norm(xi[0:3,2])
        xi[3:6,2] = xi[3:6,2] - np.dot( np.dot(xi[0:3,2].T, xi[3:6,2]) / np.dot(xi[0:3,2].T,xi[0:3,2]), xi[0:3,2])

        xi[:,3]= (xi[:,3].reshape((6,1)) + dp[18:24].reshape((6,1))).reshape((6,))
        xi[0:3,3]=xi[0:3,3] / np.linalg.norm(xi[0:3,3])
        xi[3:6,3]= xi[3:6,3] - np.dot(np.dot(xi[0:3,3].T, xi[3:6,3]) / np.dot(xi[0:3,3].T,xi[0:3,3]),xi[0:3,3])

        xi[:,4]= (xi[:,4].reshape((6,1)) + dp[24:30].reshape((6,1))).reshape((6,))
        xi[0:3,4]=xi[0:3,4] / np.linalg.norm(xi[0:3,4])
        xi[3:6,4]= xi[3:6,4] - np.dot(np.dot(xi[0:3,4].T, xi[3:6,4]) / np.dot(xi[0:3,4].T,xi[0:3,4]),xi[0:3,4])

        xi[:,5]= (xi[:,5].reshape((6,1)) + dp[30:36].reshape((6,1))).reshape((6,))
        xi[0:3,5]=xi[0:3,5] / np.linalg.norm(xi[0:3,5])
        xi[3:6,5]= xi[3:6,5] - np.dot(np.dot(xi[0:3,5].T, xi[3:6,5]) / np.dot(xi[0:3,5].T,xi[0:3,5]),xi[0:3,5])

        xist = xist + dp[36:42]

    return xi, xist
